## Implementing binary decision trees

The goal of this notebook is to implement your own binary decision tree classifier. You will:
    
* Use SFrames to do some feature engineering.
* Transform categorical variables into binary variables.
* Write a function to compute the number of misclassified examples in an intermediate node.
* Write a function to find the best feature to split on.
* Build a binary decision tree from scratch.
* Make predictions using the decision tree.
* Evaluate the accuracy of the decision tree.
* Visualize the decision at the root node.

**Important Note**: In this assignment, we will focus on building decision trees where the data contain **only binary (0 or 1) features**. This allows us to avoid dealing with:
* Multiple intermediate nodes in a split
* The thresholding issues of real-valued features.

# Fire up Graphlab Create

Make sure you have the latest version of GraphLab Create.

In [52]:
import graphlab
import numpy as np
from graphlab import SArray
from __future__ import division

# Load the lending club dataset

We will be using the same [LendingClub](https://www.lendingclub.com/) dataset as in the previous assignment.

In [53]:
loans = graphlab.SFrame('./Data/lending-club-data.gl/')

Like the previous assignment, we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [54]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

Unlike the previous assignment where we used several features, in this assignment, we will just be using 4 categorical
features: 

1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

Since we are building a binary decision tree, we will have to convert these categorical features to a binary representation in a subsequent section using 1-hot encoding.

In [55]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

Let's explore what the dataset looks like.

In [56]:
loans

grade,term,home_ownership,emp_length,safe_loans
B,36 months,RENT,10+ years,1
C,60 months,RENT,< 1 year,-1
C,36 months,RENT,10+ years,1
C,36 months,RENT,10+ years,1
A,36 months,RENT,3 years,1
E,36 months,RENT,9 years,1
F,60 months,OWN,4 years,-1
B,60 months,RENT,< 1 year,-1
C,60 months,OWN,5 years,1
B,36 months,OWN,10+ years,1


## Subsample dataset to make sure classes are balanced

Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We use `seed=1` so everyone gets the same results.

In [57]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.502236174422
Percentage of risky loans                : 0.497763825578
Total number of loans in our new dataset : 46508


**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in this [paper](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5128907&url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F69%2F5173046%2F05128907.pdf%3Farnumber%3D5128907 ). For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

## Transform categorical data into binary features

In this assignment, we will implement **binary decision trees** (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features. 

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

Since this code requires a few Python and GraphLab tricks, feel free to use this block of code as is. Refer to the API documentation for a deeper understanding.

In [58]:
loans_data = risky_loans.append(safe_loans)
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    
    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

Let's see what the feature columns look like now:

In [59]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

In [60]:
print "Number of features (after binarizing categorical variables) = %s" % len(features)

Number of features (after binarizing categorical variables) = 25


Let's explore what one of these columns looks like:

In [61]:
loans_data['grade.A']

dtype: int
Rows: 46508
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, ... ]

This column is set to 1 if the loan grade is A and 0 otherwise.

**Checkpoint:** Make sure the following answers match up.

In [62]:
print "Total number of grade.A loans : %s" % loans_data['grade.A'].sum()
print "Expexted answer               : 6422"

Total number of grade.A loans : 6422
Expexted answer               : 6422


## Train-test split + Split test data into target1 (for expansion) and target2 (for reduction)

We split the data into a train test split with 80% of the data in the training set and 20% of the data in the test set.

In [63]:
train_data, test_data = loans_data.random_split(.8, seed=1)

In [64]:
target_data1, target_data2 = test_data.random_split(.5)

### Let's convert the data into numpy arrays

In [71]:
X_source = train_data[features].to_numpy()
X_target1 = target_data1[features].to_numpy()
X_target2 = target_data2[features].to_numpy()

y_source = train_data['safe_loans'].to_numpy()
y_target1 = target_data1['safe_loans'].to_numpy()
y_target2 = target_data2['safe_loans'].to_numpy()

# Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.

## Function to count number of mistakes while predicting majority class

Recall from the lecture that prediction at an intermediate node works by predicting the **majority class** for all data points that belong to this node.

Now, we will write a function that calculates the number of **missclassified examples** when predicting the **majority class**. This will be used to help determine which feature is the best to split on at a given node of the tree.

**Note**: Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node. 

** Steps to follow **:
* ** Step 1:** Calculate the number of safe loans and risky loans.
* ** Step 2:** Since we are assuming majority class prediction, all the data points that are **not** in the majority class are considered **mistakes**.
* ** Step 3:** Return the number of **mistakes**.


Now, let us write the function `intermediate_node_num_mistakes` which computes the number of misclassified examples of an intermediate node given the set of labels (y values) of the data points contained in the node. 

In [75]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    Safe = (labels_in_node==1).sum()
    
    # Count the number of -1's (risky loans)
    Risky = (labels_in_node==-1).sum()
                
    # Return the number of mistakes that the majority classifier makes.
    if Safe > Risky:
        return Risky
    else: 
        return Safe    
        

## Function to pick best feature to split on

The function **best_splitting_feature** takes 3 arguments: 
1. The data (SFrame of data which includes all of the feature columns and label column)
2. The features to consider for splits (a list of strings of column names to consider for splits)
3. The name of the target/label column (string)

The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

Recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Follow these steps: 
* **Step 1:** Loop over each feature in the feature list
* **Step 2:** Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the **left** split), and one group where all of the data has feature value 1 or True (we will call this the **right** split). Make sure the **left** split corresponds with 0 and the **right** split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.
* **Step 3:** Calculate the number of misclassified examples in both groups of data and use the above formula to compute the **classification error**.
* **Step 4:** If the computed error is smaller than the best error found so far, store this **feature and its error**.

This may seem like a lot, but we have provided pseudocode in the comments in order to help you implement the function correctly.

**Note:** Remember that since we are only dealing with binary features, we do not have to consider thresholds for real-valued features. This makes the implementation of this function much easier.


In [96]:
# X matrix of features (p datapoints x N features)
# y vector of labels (p x 1)

def best_splitting_feature(X, y):
        
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(y))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in range(X.shape[1]):
        
        # The left split will have all data points where the feature value is 0
        ind_left = np.where(X[:,feature] == 0)
        left_split = X[X[:,feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ind_right = np.where(X[:,feature] == 1)
        right_split = X[X[:,feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        left_mistakes = intermediate_node_num_mistakes(y[ind_left])            

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(y[ind_right])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = float((left_mistakes+right_mistakes)/(num_data_points))

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error < best_error:
            best_feature = feature
            best_error = error
        
    return best_feature # Return the best feature we found

In [162]:
best_splitting_feature(np.ones([10,5]),np.array([-1,1,1,-1,1,1,1,-1,1,1]))

0

## Building the tree

With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'splitting_feature'  : The feature that this node splits on.
    }

First, we will write a function that creates a leaf node given a set of target values. Fill in the places where you find `## YOUR CODE HERE`. There are **three** places in this function for you to fill in.

In [88]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True,
            'prediction': None   }   
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] =  1     
    else:
        leaf['prediction'] = -1        
        
    # Return the leaf node        
    return leaf 

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:
1. **Stopping condition 1:** All data points in a node are from the same class.
2. **Stopping condition 2:** No more features to split on. (**Deprecated for now**)
3. **Additional stopping condition:** In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the **max_depth** of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process. 

Now, we will write down the skeleton of the learning algorithm. Fill in the places where you find `## YOUR CODE HERE`. There are **seven** places in this function for you to fill in.

In [153]:
def decision_tree_create(X, y, N_features_to_sample, current_depth = 0, max_depth = 10):

    #randomly sample a subset of features
    Nfeatures = X.shape[1]
    features = np.random.choice(Nfeatures, N_features_to_sample, replace=False)    
    
    #select only the features sampled for this run
    Xcurrent = X[:,features]
    target_values = y
    
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    print "Features selected = %s" % features
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target_values) == 0:  
        print "Stopping condition 1 reached."     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    # in this implementation we assume we can repeat the split on the same feature
    #   if len(remaining_features) == 0:   
    #       print "Stopping condition 2 reached."    
    #       # If there are no remaining features to consider, make current node a leaf node
    #       return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth:  
        print "Reached maximum depth. Stopping for now."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    splitting_feature = best_splitting_feature(Xcurrent,y)
    
    # Split on the best feature that we found. 
    ind_left = np.where(Xcurrent[:,splitting_feature] == 0)
    left_split = np.squeeze(X[ind_left,:])
    y_left = np.squeeze(y[ind_left])

    ind_right = np.where(Xcurrent[:,splitting_feature] == 1)
    right_split = np.squeeze(X[ind_right,:])
    y_right = np.squeeze(y[ind_right])
    
    #remaining_features.remove(splitting_feature) #for now we ignore the remaining features
    print y_left.shape, y_right.shape
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(y_left), len(y_right))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(y):
        print "Creating leaf node."
        return create_leaf(y_left)
    if len(right_split) == len(y):
        print "Creating leaf node."
        return create_leaf(y_right)

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, y_left, N_features_to_sample, current_depth + 1, max_depth)        
    right_tree = decision_tree_create(right_split, y_right, N_features_to_sample, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree
            }

In [117]:
a = np.array([[[[1],[2]],[[3],[4]]]])
print a.shape
b = np.squeeze(a)
print b.shape

(1, 2, 2, 1)
(2, 2)


Here is a recursive function to count the nodes in your tree:

In [90]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [91]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1 
    return count_leaves(tree['left']) + count_leaves(tree['right'])
        

## Build the tree!

Now that all the tests are passing, we will train a tree model on the **train_data**. Limit the depth to 6 (**max_depth = 6**) to make sure the algorithm doesn't run for too long. Call this tree **my_decision_tree**. 

**Warning**: This code block may take 1-2 minutes to learn. 

In [157]:
print sum(sum(X_source==1))+sum(sum(X_source==0))
print X_source.shape[0]*X_source.shape[1]

930600
930600


In [154]:
tree1 = decision_tree_create(X_source,y_source,np.sqrt(X_source.shape[1]),max_depth=20)

/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Features selected = [ 1 20 21 12 23]
(26858,) (10366,)
Split on feature 0. (26858, 10366)
--------------------------------------------------------------------
Subtree, depth = 1 (26858 data points).
Features selected = [ 1  5  7 21 17]
(8175,) (18683,)
Split on feature 2. (8175, 18683)
--------------------------------------------------------------------
Subtree, depth = 2 (8175 data points).
Features selected = [24  7  8  9  6]
(7983,) (192,)
Split on feature 0. (7983, 192)
--------------------------------------------------------------------
Subtree, depth = 3 (7983 data points).
Features selected = [14  0  7 13 11]
(7887,) (96,)
Split on feature 1. (7887, 96)
--------------------------------------------------------------------
Subtree, depth = 4 (7887 data points).
Features selected = [17  0  2 20  1]
(7388,) (499,)
Split on feature 0. (7388, 499)
------------------------------

TypeError: len() of unsized object

In [129]:
count_nodes(tree1)

19

## Making predictions with a decision tree

As discussed in the lecture, we can make predictions from the decision tree with a simple recursive function. Below, we call this function `classify`, which takes in a learned `tree` and a test point `x` to classify.  We include an option `annotate` that describes the prediction path when set to `True`.

In [132]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'],x, annotate)

#### compute the path to the leaf followed by data point x  

In [133]:
def datapath(tree, x, branch = 1):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        return branch 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]

        if split_feature_value == 0:
            
            return datapath(tree['left'], x, 2*branch)
        else:
            return datapath(tree['right'],x, 2*branch+1)

In [146]:
def evaluate_classification_error(tree, X, y):
    # Apply the classify(tree, x) to each row in your data
    prediction = map(lambda x: classify(tree,x), X)
    
    # Once you've made the predictions, calculate the classification error and return it
    mistakes = sum(prediction != y)
    error = mistakes/len(y)
    return error

In [149]:
evaluate_classification_error(tree1,X_target1,y_target1)


0.53972064600611092

#### Local expansion of the tree

In [116]:
def expansion_reduction(tree,dataT1,dataT2,features,target):

    Tree = tree #a copy of the tree
    
    leavesData1 = np.zeros(len(dataT1))
    leavesData2 = np.zeros(len(dataT2))
    
    for i in range(len(dataT1)):
        leavesData1[i]=  datapath(tree,dataT1[i]) #indicates the leaf where each data point ends up
    for i in range(len(dataT2)):
        leavesData2[i]=  datapath(tree,dataT2[i]) #indicates the leaf where each data point ends up
        
    Uleaves1 = np.unique(leavesData1)  #the path to each leaf followed by data1
    Uleaves2 = np.unique(leavesData2)  #the path to each leaf followed by data2
    Uleaves = list(set(Uleaves1) & set(Uleaves2)) #leaves reached by both data1 and data2
            
    #expanding each leaf on the 1st bootstrap replica of target data
    for i in Uleaves:
        ind_data1 = np.where(leavesData1==i) #indices of datapoints for each leaf
        print ind_data1
        Exp_tree = decision_tree_create(dataT1[ind_data1], features, 'safe_loans', max_depth = 3)

        #classification error at each leaf for Data T2
        ind_data2 = np.where(leavesData2==i)
        Err_leavesT2 = intermediate_node_num_mistakes(dataT2[ind_data2][target])/len(dataT2[ind_data2])

        #error at the current subtree on Data T2
        Err_subtreeT2 = evaluate_classification_error(Exp_tree, dataT2[ind_data2])
        
        #comparing the error of the subtree with that at the leaf node of the original tree
        if Err_subtreeT2 < Err_leavesT2:
            Tree = mergetrees(Tree,i,Exp_tree)
    
    return Tree

In [105]:
def mergetrees(tree1,leafnr,tree2):
    leafnrbin = bin(leafnr)[3:]
    path = ''
    for i in range(len(leafnrbin)):
        if leafnrbin[i] == '0':
            path=path+str("['left']")
        else:
            path=path+str("['right']") 
        print(path)
    exec ('tree1'+path+"['prediction']"+'=None')
    exec ('tree1'+path+"['is_leaf']"+'=False')
    exec ('tree1'+path+"['left']"+"=tree2['left']")
    exec ('tree1'+path+"['right']"+"=tree2['right']")
    exec ('tree1'+path+"['splitting_feature']"+"=tree2['splitting_feature']")
    
#    print ('tree1'+path+"['prediction']"+'=None')
#    print ('tree1'+path+"['is_leaf']"+'=False')
#    print ('tree1'+path+"['left']"+"=tree2['left']")
#    print ('tree1'+path+"['right']"+"=tree2['right']")
#    print ('tree1'+path+"['splitting_feature']"+"=tree2['splitting_feature']")
    
    #print('tree1'+path+'=tree2')
    return tree1
    

In [123]:
t1 = target_data1.to_numpy()
print t1

[[-1  0  0 ...,  0  0  0]
 [-1  0  1 ...,  0  0  0]
 [-1  0  0 ...,  0  0  0]
 ..., 
 [ 1  0  0 ...,  0  0  0]
 [ 1  0  0 ...,  0  0  0]
 [ 1  0  0 ...,  0  0  0]]


In [117]:
expansion_reduction(tree1,target_data1,target_data2,features,target)

TypeError: Invalid key type: must be str, bytes or type

In [106]:
tree1 = mergetrees(tree1,2,tree2)

['left']


In [108]:
count_leaves(tree2)

7

In [109]:
leaves = np.zeros(len(test_data))
for i in range(len(test_data)):
    leaves[i]=  datapath(my_decision_tree,test_data[i])
    
print np.unique(leaves)
print count_nodes(my_decision_tree)


[ 4.  5.  6.  7.]
7


In [180]:
my_decision_tree['left']['left']['left']

{'is_leaf': False,
 'left': {...},
 'prediction': None,
 'right': {'is_leaf': True,
  'left': None,
  'prediction': 1,
  'right': None,
  'splitting_feature': None},
 'splitting_feature': 'term. 36 months'}

In [78]:
leaves

array([ 4.,  6.,  4., ...,  7.,  6.,  6.])

In [119]:
count_leaves(my_decision_tree)

4

In [120]:
count_nodes(my_decision_tree)

7

Now, let's consider the first example of the test set and see what `my_decision_tree` model predicts for this data point.

In [84]:
test_data[0]

{'emp_length.1 year': 0,
 'emp_length.10+ years': 0,
 'emp_length.2 years': 1,
 'emp_length.3 years': 0,
 'emp_length.4 years': 0,
 'emp_length.5 years': 0,
 'emp_length.6 years': 0,
 'emp_length.7 years': 0,
 'emp_length.8 years': 0,
 'emp_length.9 years': 0,
 'emp_length.< 1 year': 0,
 'emp_length.n/a': 0,
 'grade.A': 0,
 'grade.B': 0,
 'grade.C': 0,
 'grade.D': 1,
 'grade.E': 0,
 'grade.F': 0,
 'grade.G': 0,
 'home_ownership.MORTGAGE': 0,
 'home_ownership.OTHER': 0,
 'home_ownership.OWN': 0,
 'home_ownership.RENT': 1,
 'safe_loans': -1,
 'term. 36 months': 0,
 'term. 60 months': 1}

In [85]:
print 'Predicted class: %s ' % classify(my_decision_tree, test_data[0])

Predicted class: -1 


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [86]:
classify(my_decision_tree, test_data[0], annotate=True)

Split on term. 36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
Split on grade.D = 1
At leaf, predicting -1


-1

** Quiz question:** What was the feature that **my_decision_tree** first split on while making the prediction for test_data[0]?

** Quiz question:** What was the first feature that lead to a right split of test_data[0]?

** Quiz question:** What was the last feature split on before reaching a leaf node for test_data[0]?

## Evaluating your decision tree

Now, we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset.

Again, recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Now, write a function called `evaluate_classification_error` that takes in as input:
1. `tree` (as described above)
2. `data` (an SFrame)

This function should return a prediction (class label) for each row in `data` using the decision `tree`. Fill in the places where you find `## YOUR CODE HERE`. There is **one** place in this function for you to fill in.

In [116]:
from __future__ import division
print test_data[target][0:10]
prediction = graphlab.SArray([1,-1,-1,1,-1,1,1,1,1,1])
print prediction
errors = (prediction != test_data[target][0:10]).sum()
print errors
classerr = errors/10
print classerr

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[1, -1, -1, 1, -1, 1, 1, 1, 1, 1]
7
0.7


In [117]:
from __future__ import division

def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))
    
    # Once you've made the predictions, calculate the classification error and return it
    mistakes = (prediction != data[target]).sum()
    error = mistakes/len(data[target])
    return error

Now, let's use this function to evaluate the classification error on the test set.

In [118]:
evaluate_classification_error(my_decision_tree, test_data)

0.3837785437311504

**Quiz Question:** Rounded to 2nd decimal point, what is the classification error of **my_decision_tree** on the **test_data**?

Error = 0.38

## Printing out a decision stump

As discussed in the lecture, we can print out a single decision stump (printing out the entire tree is left as an exercise to the curious reader). 

In [119]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print "(leaf, label: %s)" % tree['prediction']
        return None
    split_feature, split_value = split_name.split('.')
    print '                       %s' % name
    print '         |---------------|----------------|'
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '  [{0} == 0]               [{0} == 1]    '.format(split_name)
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree'))

In [120]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


**Quiz Question:** What is the feature that is used for the split at the root node?

### Exploring the intermediate left subtree

The tree is a recursive dictionary, so we do have access to all the nodes! We can use
* `my_decision_tree['left']` to go left
* `my_decision_tree['right']` to go right

* Term 36M

In [121]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


### Exploring the left subtree of the left subtree


In [122]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade.A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.B == 0]               [grade.B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


**Quiz question:** What is the path of the **first 3 feature splits** considered along the **left-most** branch of **my_decision_tree**?

Term 36M -> grade A -> grade B

**Quiz question:** What is the path of the **first 3 feature splits** considered along the **right-most** branch of **my_decision_tree**?

In [128]:
print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.D == 0]               [grade.D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


term 36M -> grade D